In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from transformers import AutoTokenizer, AutoModel
import torch

data = pd.read_csv("emails.csv")
texts = data['text'].tolist()
labels = data['label']
label_map = {'legitimate': 0, 'phishing': 1}
numeric_labels = [label_map[l] for l in labels]

X_train, X_test, y_train, y_test = train_test_split(texts, numeric_labels, test_size=0.5, random_state=42, stratify=numeric_labels)

# Train baseline model
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

nb_model = MultinomialNB()
nb_model.fit(X_train_counts, y_train)

# Prepare LLM embeddings
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModel.from_pretrained(model_name)

def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = llm_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

X_train_emb = get_embeddings(X_train)
X_test_emb = get_embeddings(X_test)

llm_clf = LogisticRegression()
llm_clf.fit(X_train_emb, y_train)

# Combined decision
nb_probs = nb_model.predict_proba(X_test_counts)
llm_predictions = llm_clf.predict(X_test_emb)

final_predictions = []
for i, probs in enumerate(nb_probs):
    phishing_prob = probs[1]  # Probability of phishing
    if phishing_prob > 0.9:
        final_predictions.append(1)
    elif phishing_prob < 0.1:
        final_predictions.append(0)
    else:
        final_predictions.append(llm_predictions[i])

y_pred = final_predictions  # Use final_predictions as y_pred

print("Combined Pipeline Results:")
print(classification_report(y_test, y_pred, target_names=["legitimate", "phishing"]))

Combined Pipeline Results:
              precision    recall  f1-score   support

  legitimate       1.00      1.00      1.00         4
    phishing       1.00      1.00      1.00         3

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7



In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from transformers import AutoTokenizer, AutoModel
import torch, numpy as np

# ------------------ 1. Load data -----------------
df = pd.read_csv("emails_from_spamassassin.csv")
texts  = df["text"].tolist()
labels = df["label"].map({"legitimate":0, "phishing":1}).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.5, random_state=42, stratify=labels)

# ------------------ 2. Baseline NB ---------------
vectorizer = CountVectorizer()
Xtr_counts = vectorizer.fit_transform(X_train)
Xte_counts = vectorizer.transform(X_test)

nb_model = MultinomialNB().fit(Xtr_counts, y_train)

# ------------------ 3. LLM setup -----------------
model_name = "distilbert-base-uncased"
tokenizer   = AutoTokenizer.from_pretrained(model_name)
llm_model   = AutoModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
llm_model.to(device)

def embed(batch_texts, batch_size=32, max_len=256):
    """Return (N, 768) numpy matrix of mean-pooled embeddings."""
    all_vecs = []
    for i in range(0, len(batch_texts), batch_size):
        batch = batch_texts[i:i+batch_size]
        tok = tokenizer(batch,
                        return_tensors="pt",
                        padding=True,
                        truncation=True,
                        max_length=max_len).to(device)
        with torch.no_grad():
            hidden = llm_model(**tok).last_hidden_state  # (b, seq, 768)
            vecs   = hidden.mean(dim=1).cpu()            # (b, 768)
        all_vecs.append(vecs)
    return torch.cat(all_vecs, dim=0).numpy()

X_train_emb = embed(X_train, batch_size=32)
X_test_emb  = embed(X_test,  batch_size=32)

llm_clf = LogisticRegression(max_iter=1000).fit(X_train_emb, y_train)

# ------------------ 4. Two-stage decision --------
nb_probs       = nb_model.predict_proba(Xte_counts)
llm_pred       = llm_clf.predict(X_test_emb)
final_pred = []

for prob, fallback in zip(nb_probs, llm_pred):
    p = prob[1]              # phishing probability
    if p > 0.9:   final_pred.append(1)
    elif p < 0.1: final_pred.append(0)
    else:         final_pred.append(fallback)

print("Combined Pipeline Results:\n")
print(classification_report(y_test, final_pred,
                            target_names=["legitimate", "phishing"]))


Combined Pipeline Results:

              precision    recall  f1-score   support

  legitimate       0.98      0.99      0.99      2202
    phishing       0.99      0.96      0.98      1200

    accuracy                           0.98      3402
   macro avg       0.98      0.98      0.98      3402
weighted avg       0.98      0.98      0.98      3402

